In [1]:
from datetime import datetime, timedelta
import geojson
import geopandas as gpd
import numpy as np
import os
from pathlib import Path
import pandas as pd
from pyproj import Transformer, CRS
import random
import rasterio as rio
from rasterio.mask import mask
from shapely.geometry import Polygon
from shapely.geometry import shape
import shutil
import subprocess
import time

# IMPORT FUNCTIONS --------------------------------------------------------
from ff_file_generator_automatic import create_ff_file
from ffgeojsonTogeojson import process_ffgeojson_files

# Import the relevant functions from extend.py
from extend import get_utm_crs, process_raster_files

# Import the creation of the cfg file for windNinja
from genWindNinjaFile_automatic import create_config_file
# -------------------------------------------------------------------------


# .........................
start_time = time.time()# .
# .........................


In [2]:
# Parent directory where all the subfolders are located
DEM_path = '/home/jsoma/europeData/EU-DEM.tif'
CLC_path = '/home/jsoma/europeData/EU-CLC_v2018.tif'
parent_dir = Path('/home/jsoma/examples/simulation_crete')


# Iterate through each subdirectory in the parent directory
for subdir in parent_dir.iterdir():

    print(f"Processing folder: {subdir}")
    run_dir = str(subdir)

    # Change to the folder
    os.chdir(subdir)
    # Find the file that ends with '_data.csv'
    wind_data_file = next((file for file in os.listdir('.') if file.endswith('_data.csv')), None)
    wind_data = pd.read_csv(wind_data_file)
    num_steps = np.shape(wind_data)[0]

    # Parameters -------------------------------------------------------
    # Get the common data
    common_data = wind_data.iloc[[0]]
    lat = common_data['lat'].iloc[0]
    lon = common_data['lon'].iloc[0]
    print(lat)
    size_km1 = 90  # Size of the square in kilometers
    size_km2 = 50

    # Get the UTM CRS
    dstCrs_UCTM = get_utm_crs(lon, lat)
    output_dir = os.getcwd()
    elevation_path, landcover_path = process_raster_files(lon, lat, size_km1, size_km2, DEM_path, CLC_path, output_dir, dstCrs_UCTM)
    
    # set randomly the maximun area of the fire
    # area_max = random.choice([500, 1000, 1500, 2000, 2500])
    area_max = np.random.uniform(500, 3000)
    print(f'The maximum extension of this fire is: {area_max} ha')
    
    # loop over the rows of wind_data to run the simulation
    # for i in range(num_steps):
    for i, row in wind_data.iterrows():

        folderName = f't{i}'

        if not os.path.exists(folderName):
            os.mkdir(folderName)
            shutil.copy('elevation.tif', folderName)  # Copy elevation file for all folders

            if i == 0:
                shutil.copy('land_cover.tif', folderName)  # Copy land_cover.tif only for t0
            else: 
                # Modify land cover based on previous step's GeoJSON output
                print(f"Attempting to read file: {geojson_file}")
                land_cover = os.path.join(subdir, 'land_cover.tif')
                out_forefire_geojson = gpd.read_file(geojson_file)

                with rio.open(land_cover) as src:
                    land_cover_array = src.read(1)
                    land_cover_array_copy = land_cover_array.copy()
                    land_cover_crs = src.crs

                    if out_forefire_geojson.crs != land_cover_crs:
                        out_forefire_geojson = out_forefire_geojson.to_crs(land_cover_crs)

                    geometries = [geom for geom in out_forefire_geojson.geometry]
                    out_image, out_transform = mask(src, geometries, crop=False, nodata=-1, filled=True)

                    land_cover_array_copy[out_image[0] != -1] = 324  # Set burned areas to specific value
                    land_cover_array_copy = land_cover_array_copy.astype(np.int16)

                    out_meta = src.meta.copy()

                    # Save modified land cover for the next iteration
                    output_path = os.path.join(folderName, 'land_cover.tif')
                    with rio.open(output_path, 'w', **out_meta) as dest:
                        dest.write(land_cover_array_copy, 1)
    
        
        os.chdir(folderName)
        print('-------------------')
        print(folderName)
        print('-------------------')
        # Store teh variables for this time-step
        wind_speed =  round(wind_data['wind_speed'][i], 1)
        wind_direction = round(wind_data['wind_direction'][i])
        date =  wind_data['date'][i]

        # 3) WindNinja Simulation  --------------------
        # Gen cfg file
        create_config_file(wind_speed, wind_direction)
        # run WindNinja
        subprocess.run(['WindNinja_cli', 'wind_domain_avg.cfg'])

        # 4) Create landscape.nc file once WindNinja is done --------
        subprocess.run(['python3', '/home/jsoma/scripts/scripts/allTogether/createLandscape.py'])

        # 5) Create a tX.ff file -------------------------------------
        
        create_ff_file(i, folderName, run_dir, date)
        
        # 6) Run ForeFire -------------------------------------------
        subprocess.run(['forefire', '-i', f't{i}.ff'], check=True)

        # 7) Proccess the output of ForeFire -----------------------
        process_ffgeojson_files()
        geojson_file = os.path.join(run_dir, f't{i}', f't{i}.geojson')

        # 8) Prepare for the next iteration ---------------------------
        # Break if the geojson file is bigger than 2500 ha
        geojson_file_step = gpd.read_file(geojson_file)
        geojson_file_step = geojson_file_step.to_crs("EPSG:3857")  # Reproject to a projected CRS (e.g., Web Mercator)
        geojson_file_step['area_ha'] = geojson_file_step.geometry.area/10_000 #calculate the ha of the fire
        print(geojson_file_step['area_ha'])
        if (geojson_file_step['area_ha'] > area_max).any():
            print(f"Stopping simulation at t{i} because an area exceeds 2500 ha.")
            final_name = os.path.join(run_dir, f't{i}', f'final_t{i}.geojson')
            os.rename(geojson_file, final_name)
            break

        # break if time is complete
        os.chdir(subdir)
        if i == num_steps - 1:
            final_name = os.path.join(run_dir, f't{i}', f'final_t{i}.geojson')
            os.rename(geojson_file, final_name)
            break
    


    # Change back to the parent directory
    os.chdir(parent_dir)

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Total elapsed time: {elapsed_time / 60:.2f} minutes")

Processing folder: /home/jsoma/examples/simulation_crete/2012-08-29_25.31E_35.26N
35.2593
UTM Zone: 35, Hemisphere: N, EPSG Code: EPSG:32635
The maximum extension of this fire is: 1500 ha
-------------------
t0
-------------------
Configuration file 'wind_domain_avg.cfg' has been created successfully.
Run 0: Reading elevation file...
Run 0: Run number 0 started with 8 threads.
Run 0: Generating mesh...
Run 0: Initializing flow...
Run 0: Building equations...
Run 0: Solving...
Run 0 (solver): 13% complete
Run 0 (solver): 31% complete
Run 0 (solver): 59% complete
Run 0 (solver): 80% complete
Run 0 (solver): 91% complete
Run 0 (solver): 98% complete
Run 0 (solver): 100% complete
Run 0: Writing output files...
Run 0: Meshing time was 0.059640 seconds.
Run 0: Initialization time was 0.109063 seconds.
Run 0: Equation building time was 0.901552 seconds.
Run 0: Solver time was 8.466345 seconds.
Run 0: Output writing time was 0.048668 seconds.
Run 0: Total simulation time was 10.160218 seconds.

ERROR 1: Operation timed out after 5001 milliseconds with 0 out of 0 bytes received


Run 0: Reading elevation file...
Run 0: Run number 0 started with 8 threads.
Run 0: Generating mesh...
Run 0: Initializing flow...
Run 0: Building equations...
Run 0: Solving...
Run 0 (solver): 12% complete
Run 0 (solver): 45% complete
Run 0 (solver): 66% complete
Run 0 (solver): 83% complete
Run 0 (solver): 93% complete
Run 0 (solver): 99% complete
Run 0 (solver): 100% complete
Run 0: Writing output files...
Run 0: Meshing time was 0.063607 seconds.
Run 0: Initialization time was 0.129586 seconds.
Run 0: Equation building time was 0.765200 seconds.
Run 0: Solver time was 8.406572 seconds.
Run 0: Output writing time was 0.034326 seconds.
Run 0: Total simulation time was 9.930192 seconds.
Run 0: Run number 0 done!
EPSG: 32635
writing landscape.nc
Rothermel
loading at time 2016-10-25_18:00:00Z yday
Temperature variable not found in the NetCDF file.
{"type": "FeatureCollection",
	"projection":"EPSG:32635",
	"features": [
		{"type": "Feature",
			"properties": {},
			"geometry": {
				"typ

ERROR 1: Operation timed out after 5001 milliseconds with 0 out of 0 bytes received


Run 0: Reading elevation file...
Run 0: Run number 0 started with 8 threads.
Run 0: Generating mesh...
Run 0: Initializing flow...
Run 0: Building equations...
Run 0: Solving...
Run 0 (solver): 14% complete
Run 0 (solver): 36% complete
Run 0 (solver): 63% complete
Run 0 (solver): 80% complete
Run 0 (solver): 92% complete
Run 0 (solver): 99% complete
Run 0 (solver): 100% complete
Run 0: Writing output files...
Run 0: Meshing time was 0.055155 seconds.
Run 0: Initialization time was 0.109343 seconds.
Run 0: Equation building time was 0.819119 seconds.
Run 0: Solver time was 8.565934 seconds.
Run 0: Output writing time was 0.061033 seconds.
Run 0: Total simulation time was 10.247238 seconds.
Run 0: Run number 0 done!
EPSG: 32635
writing landscape.nc
Rothermel
loading at time 2009-07-14_12:00:00Z yday
Temperature variable not found in the NetCDF file.
{"type": "FeatureCollection",
	"projection":"EPSG:32635",
	"features": [
		{"type": "Feature",
			"properties": {},
			"geometry": {
				"ty

...

Run 0: Reading elevation file...
Run 0: Run number 0 started with 8 threads.
Run 0: Generating mesh...
Run 0: Initializing flow...
Run 0: Building equations...
Run 0: Solving...
Run 0 (solver): 14% complete
Run 0 (solver): 38% complete
Run 0 (solver): 66% complete
Run 0 (solver): 81% complete
Run 0 (solver): 92% complete
Run 0 (solver): 98% complete
Run 0 (solver): 100% complete
Run 0: Writing output files...
Run 0: Meshing time was 0.062500 seconds.
Run 0: Initialization time was 0.127547 seconds.
Run 0: Equation building time was 0.781510 seconds.
Run 0: Solver time was 7.857085 seconds.
Run 0: Output writing time was 0.029324 seconds.
Run 0: Total simulation time was 9.332101 seconds.
Run 0: Run number 0 done!
EPSG: 32635
writing landscape.nc
Rothermel
loading at time 2013-07-27_12:00:00Z yday
Temperature variable not found in the NetCDF file.
{"type": "FeatureCollection",
	"projection":"EPSG:32635",
	"features": [
		{"type": "Feature",
			"properties": {},
			"geometry": {
				"typ

ERROR 1: Operation timed out after 5001 milliseconds with 0 out of 0 bytes received


Run 0: Reading elevation file...
Run 0: Run number 0 started with 8 threads.
Run 0: Generating mesh...
Run 0: Initializing flow...
Run 0: Building equations...
Run 0: Solving...
Run 0 (solver): 14% complete
Run 0 (solver): 37% complete
Run 0 (solver): 65% complete
Run 0 (solver): 81% complete
Run 0 (solver): 92% complete
Run 0 (solver): 98% complete
Run 0 (solver): 100% complete
Run 0: Writing output files...
Run 0: Meshing time was 0.066173 seconds.
Run 0: Initialization time was 0.126929 seconds.
Run 0: Equation building time was 0.794172 seconds.
Run 0: Solver time was 8.064475 seconds.
Run 0: Output writing time was 0.034190 seconds.
Run 0: Total simulation time was 9.873828 seconds.
Run 0: Run number 0 done!
EPSG: 32635
writing landscape.nc
Rothermel
loading at time 2013-07-27_15:00:00Z yday
Temperature variable not found in the NetCDF file.
{"type": "FeatureCollection",
	"projection":"EPSG:32635",
	"features": [
		{"type": "Feature",
			"properties": {},
			"geometry": {
				"typ

ERROR 1: Operation timed out after 5001 milliseconds with 0 out of 0 bytes received


Run 0: Reading elevation file...
Run 0: Run number 0 started with 8 threads.
Run 0: Generating mesh...
Run 0: Initializing flow...
Run 0: Building equations...
Run 0: Solving...
Run 0 (solver): 16% complete
Run 0 (solver): 39% complete
Run 0 (solver): 66% complete
Run 0 (solver): 81% complete
Run 0 (solver): 93% complete
Run 0 (solver): 99% complete
Run 0 (solver): 100% complete
Run 0: Writing output files...
Run 0: Meshing time was 0.056670 seconds.
Run 0: Initialization time was 0.137444 seconds.
Run 0: Equation building time was 0.890834 seconds.
Run 0: Solver time was 7.353500 seconds.
Run 0: Output writing time was 0.023405 seconds.
Run 0: Total simulation time was 8.898618 seconds.
Run 0: Run number 0 done!
EPSG: 32635
writing landscape.nc
Rothermel
loading at time 2016-07-30_15:00:00Z yday
Temperature variable not found in the NetCDF file.
{"type": "FeatureCollection",
	"projection":"EPSG:32635",
	"features": [
		{"type": "Feature",
			"properties": {},
			"geometry": {
				"typ